# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=29)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [3]:
bootstrap_idx(X_train)

array([109,  83,  41,  84, 102, 107,  40,  66,  63,  20,  66,  64,  73,
        65,  46,  63,  55,   0,  75,   9,  59,  18,  90,  14,  62,  20,
        85,   0,  43, 104, 104,  75,  59,  40,  91,  10,  40,  31,   5,
        93,  25,  46,  41,   2,  30,  28,  58, 104,  17,  45,  30,   5,
       104,  72,  66,  11,  94,  48,  78,  51, 108,  55,  60,  63,   9,
       108,  77,  73,  49,  25,  56,  66,  10,   0,  31,  95,  93,  29,
        84, 100,   4,   8,   5,  77,  78,  47,  76,  81,  18,  87,  99,
        23,  34,  25,  70,  29,  53,  53,  87,  73,  31, 103,  64,  83,
       111,  48,   5,  54,  79,  14,  39,  83])

In [4]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({np.int64(72): 4,
         np.int64(52): 4,
         np.int64(36): 4,
         np.int64(1): 3,
         np.int64(6): 3,
         np.int64(62): 3,
         np.int64(56): 3,
         np.int64(60): 2,
         np.int64(82): 2,
         np.int64(46): 2,
         np.int64(111): 2,
         np.int64(101): 2,
         np.int64(67): 2,
         np.int64(34): 2,
         np.int64(92): 2,
         np.int64(80): 2,
         np.int64(4): 2,
         np.int64(79): 2,
         np.int64(19): 2,
         np.int64(44): 2,
         np.int64(97): 2,
         np.int64(69): 2,
         np.int64(95): 2,
         np.int64(29): 2,
         np.int64(5): 2,
         np.int64(54): 2,
         np.int64(91): 1,
         np.int64(71): 1,
         np.int64(27): 1,
         np.int64(90): 1,
         np.int64(83): 1,
         np.int64(2): 1,
         np.int64(88): 1,
         np.int64(102): 1,
         np.int64(12): 1,
         np.int64(94): 1,
         np.int64(66): 1,
         np.int64(50): 1,
         np.in

In [5]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [6]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [7]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({np.int64(0): 38, np.int64(1): 37, np.int64(2): 37})
Classes distribution in the bootstrap: Counter({np.int64(0): 43, np.int64(1): 41, np.int64(2): 28})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [ ]:
# 1. Create serval DecisionTreeClassifier and store them in a list.
from sklearn.tree import DecisionTreeClassifier
n_trees = 5
trees = []
#  2. loop and fit them by generating bootstrap samples
for _ in range(n_trees):
    tree = DecisionTreeClassifier(random_state=29)
    X_sample, y_sample = bootstrap_sample(X_train, y_train)
    tree.fit(X_sample, y_sample)
    trees.append(tree)

# 3. To predict these treess on x_test
from scipy.stats import mode
predictions = np.zeros((X_test.shape[0], n_trees), dtype=int)
for i, tree in enumerate(trees):    # enumerate是一个迭代器，每一步返回一个索引和值的元组 (index, value)
    predictions[:, i] = tree.predict(X_test)
# 4. Aggregate the predictions by majority voting
y_pred, _ = mode(predictions, axis=1)
y_pred = y_pred.ravel() 
# 5. Finally, evaluate the accuracy of the ensemble model
from sklearn.metrics import accuracy_score  
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble model accuracy: {accuracy:.2f}') 

Ensemble model accuracy: 0.95


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [9]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(
    estimator=DecisionTreeClassifier())
clf.fit(X_train, y_train).score(X_test, y_test)

0.9473684210526315

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [10]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test)

0.9473684210526315

In [11]:
from figures import plot_forest_interactive
plot_forest_interactive()

interactive(children=(IntSlider(value=0, description='max_depth', max=8), Output()), _dom_classes=('widget-int…